In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from deep_convnet import DeepConvNet
%matplotlib inline

from common.functions import softmax

import time

Using TensorFlow backend.


In [2]:
def generate_stack_list(x_list, t_list, max_n):
    stack_x_list = []
    stack_i_list = []
    for _ in range(10):
        stack_x_list.append([])
        stack_i_list.append([])
        
    fill_list = []
    for i, t in enumerate(t_list):
        if (not t in fill_list) and len(stack_x_list[t]) < max_n:
            stack_x_list[t].append(x_list[i].flatten())
            stack_i_list[t].append(i)
        else:
            fill_list.append(t)
            
    return np.array(stack_x_list), np.array(stack_i_list)

In [3]:
def generate_stack_list_with_unknown(x_list, t_list, max_n, max_unknown_n):
    stack_x_list = []
    stack_i_list = []
    stack_unknown_list = []
    for _ in range(10):
        stack_x_list.append([])
        stack_i_list.append([])
        stack_unknown_list.append([])
        
    for i, t in enumerate(t_list):
        if len(stack_x_list[t]) < max_n:
            stack_x_list[t].append(x_list[i].flatten())
            stack_i_list[t].append(i)
        elif len(stack_unknown_list[t]) < max_unknown_n:
            stack_unknown_list[t].append(x_list[i])
            
    return np.array(stack_x_list), np.array(stack_i_list), np.array(stack_unknown_list)

In [4]:
def generate_avg_list(stack_x_list):            
    avg_list = []
    for stacks in stack_x_list:
        avg = np.mean(stacks, axis=0)
        avg_list.append(avg.reshape(1,28,28))
        
    return np.array(avg_list)

In [5]:
def show_pred(x, network):
    pre = network.predict(x.reshape(1,1,28,28))
    pre_label = np.argmax(pre)
    pre_score = round(max(softmax(pre[0])), 2)

    plt.imshow(x.reshape(28,28), 'gray')
    plt.title(f"{pre_label} : {pre_score}", fontsize=20)
    plt.show()

In [6]:
def generate_setsudo(x, label, network, eps):
    d, g = network.gradient_for_fgsm(x.reshape(1,1,28,28), np.array([label]))
    p = eps * np.sign(d)
    
    return p.reshape(28,28)

In [7]:
def attack_fgsm(x, label, network, eps):
    x = x.reshape(28,28)
    p = generate_setsudo(x, label, network, eps)
    adv = (x + p).clip(min=0, max=1)
    
    return adv.reshape(1,28,28)

In [8]:
def attack_fgsm_for_stack_list(stack_x_list, network, eps):
    stack_adv_list = []
    for label, stacks in enumerate(stack_x_list):
        adv_list = []
        for x in stacks:            
#             show_pred(x, network)
            adv = attack_fgsm(x, label, network, eps)
#             show_pred(adv, network)
            adv_list.append(adv)
            
        stack_adv_list.append(adv_list)
        
    return np.array(stack_adv_list)

In [9]:
def attack_fgsm_for_stack_unknown_list(stack_unknown_list, stack_x_list, network, eps):
    stack_adv_list = []
    for label, stacks in enumerate(stack_x_list):
        adv_list = []
#         show_pred(stacks[0], network)
        p = generate_setsudo(stacks[0], label, network, eps)
#         show_pred(p, network)
        for x in stack_unknown_list[label]:            
#             show_pred(x, network)
            adv = (x.reshape(28,28) + p).clip(min=0, max=1)
#             show_pred(adv, network)
            adv_list.append(adv.reshape(1,28,28))
            
        stack_adv_list.append(adv_list)
        
    return np.array(stack_adv_list)

In [10]:
def attack_random_for_stack_list(stack_x_list, network, eps):
    random_noise = np.random.normal(
        loc   = 0,      # 平均
        scale = 1,      # 標準偏差
        size  = (28, 28),# 出力配列のサイズ
    )
    
#     plt.hist(random_noise,bins=5)
#     plt.xlim(-5,5)
    
    stack_adv_list = []
    for label, stacks in enumerate(stack_x_list):
        advs = (stack_x_list[label].reshape(-1, 1, 28, 28) + np.sign(random_noise) * eps).clip(min=0, max=1)
        stack_adv_list.append(advs)
            
    return np.array(stack_adv_list)

In [11]:
def acc_stack_list(stack_x_list, network):
    for label, stacks in enumerate(stack_x_list):
        acc = 0
        for x in stacks:
#             show_pred(stacks[0], network)
            pre = network.predict(x.reshape(1,1,28,28))
            pre_label = np.argmax(pre)
            if pre_label == label:
                acc += 1
        print(label, acc/len(stacks))
        print("-----")

In [12]:
def attack_avg_for_stack_list(stack_x_list, network, eps):
    stack_adv_list = []
    stack_avg_list = generate_avg_list(stack_x_list)
    for label, avg in enumerate(stack_avg_list):
        p = generate_setsudo(avg, label, network, eps)
        advs = (stack_x_list[label].reshape(-1, 1, 28, 28) + p).clip(min=0, max=1)
        stack_adv_list.append(advs)
        
    return np.array(stack_adv_list)

In [13]:
def attack_avg_for_stack_unknown_list(stack_unknown_list, stack_x_list, network, eps):
    stack_adv_list = []
    stack_avg_list = generate_avg_list(stack_x_list)
    for label, avg in enumerate(stack_avg_list):
        p = generate_setsudo(avg, label, network, eps)
        advs = (stack_unknown_list[label].reshape(-1, 1, 28, 28) + p).clip(min=0, max=1)
        stack_adv_list.append(advs)
        
    return np.array(stack_adv_list)

In [14]:
network = DeepConvNet()

network.load_params("deep_convnet_params.pkl")
print("loaded Network Parameters!")

loaded Network Parameters!


In [15]:
eps = 0.3

In [16]:
n = 2

In [17]:
# データの読み込み
(x_train, t_train), (x_test, t_test) = mnist.load_data()

In [18]:
stack_x_list, stack_i_list, stack_unknown_list = generate_stack_list_with_unknown(x_test, t_test, n, 100) 

In [19]:
stack_x_list = stack_x_list.astype(np.float32) / 255.0
stack_x_list.shape

(10, 2, 784)

In [20]:
stack_unknown_list.shape

(10, 100, 28, 28)

In [21]:
stack_unknown_list = stack_unknown_list.astype(np.float32) / 255.0
stack_unknown_list.shape

(10, 100, 28, 28)

In [22]:
start = time.time()
stack_adv_list = attack_fgsm_for_stack_unknown_list(stack_unknown_list, stack_x_list, network, eps)
elapsed_time = time.time() - start
print (f"fgsm : {elapsed_time}[sec]")

fgsm : 0.1695880889892578[sec]


In [23]:
start = time.time()
stack_avg_adv_list = attack_avg_for_stack_unknown_list(stack_unknown_list, stack_x_list, network, eps)
elapsed_time = time.time() - start
print (f"avg_fgsm : {elapsed_time}[sec]")

avg_fgsm : 0.15660500526428223[sec]


In [24]:
start = time.time()
stack_random_list = attack_random_for_stack_list(stack_unknown_list, network, eps)
elapsed_time = time.time() - start
print (f"random : {elapsed_time}[sec]")

random : 0.01024174690246582[sec]


In [25]:
print("normal")
acc_stack_list(stack_unknown_list.reshape(10, -1, 1, 28, 28), network)

normal
0 1.0
-----
1 0.98
-----
2 0.99
-----
3 1.0
-----
4 0.97
-----
5 0.98
-----
6 0.98
-----
7 1.0
-----
8 0.98
-----
9 1.0
-----


In [26]:
print("random")
acc_stack_list(stack_random_list, network)

random
0 0.97
-----
1 0.24
-----
2 1.0
-----
3 0.99
-----
4 0.92
-----
5 0.91
-----
6 0.95
-----
7 0.91
-----
8 0.94
-----
9 0.95
-----


In [27]:
print("FGSM")
acc_stack_list(stack_adv_list, network)

FGSM
0 0.56
-----
1 0.33
-----
2 0.96
-----
3 1.0
-----
4 0.87
-----
5 0.9
-----
6 0.86
-----
7 0.69
-----
8 0.8
-----
9 0.6
-----


In [28]:
print("avg_fgsm")
acc_stack_list(stack_avg_adv_list, network)

avg_fgsm
0 0.82
-----
1 0.46
-----
2 0.97
-----
3 0.98
-----
4 0.75
-----
5 0.99
-----
6 0.84
-----
7 0.65
-----
8 0.79
-----
9 0.83
-----
